# Copulas Time varying

In [8]:
import numpy as np
from scipy.stats import kendalltau


def gumbel_tv_logl(psi: np.ndarray, data: np.ndarray) -> float:
    """
    Computes the negative log likelihood of a time varying Gumbel copula.

    Parameters:
    psi (np.ndarray): The time-varying copula parameters.
    data (np.ndarray): The bivariate data.

    Returns:
    float: The negative log likelihood.
    """
    u, v = data[:, 0], data[:, 1]
    theta = gumbel_tv(psi, data)
    tu = (-np.log(u)) ** theta
    tv = (-np.log(v)) ** theta
    out1 = np.exp(-(tu + tv) ** (1 / theta))
    out2 = (u * v) ** (-1)
    out3 = (tu + tv) ** (-2 + 2 / theta)
    out4 = (np.log(u) * np.log(v)) ** (theta - 1)
    out5 = 1 + (theta - 1) * (tu + tv) ** (-1 / theta)
    out = out1 * out2 * out3 * out4 * out5
    print("outs:", u, v, theta)

    ll = np.sum(np.log(out))
    return -ll


def gumbel_tv(theta: np.ndarray, data: np.ndarray) -> np.ndarray:
    """
    Computes the time-varying copula dependence parameter.

    Parameters:
    theta (np.ndarray): The copula parameter vector.
    data (np.ndarray): The bivariate data.

    Returns:
    np.ndarray: The time-varying copula dependence parameter.
    """
    u, v = data[:, 0], data[:, 1]
    t = data.shape[0]
    tau = np.ones(t)
    psi = np.zeros(t)
    tau[0] = kendalltau(u, v)[0]

    for i in range(1, t):
        if i <= 10:
            psi[i] = theta[0] + theta[1] * psi[i - 1] + theta[2] * np.mean(
                np.abs(u[:i] - v[:i])
            )
        else:
            psi[i] = theta[0] + theta[1] * psi[i - 1] + theta[2] * np.mean(
                np.abs(u[i - 10 : i] - v[i - 10 : i])
            )
        tau[i] = 0.0001 + 0.75 / (1 + np.exp(-psi[i]))

    print("tau=",tau)
    psi = 1 / (1 - tau)
    return psi


# Test cases
def test_gumbel_tv_logl():
    # Simple case with known parameters
    psi = np.array([0.5])
    data = np.array([[0.5, 0.6]])
    expected_ll = -0.1653408
    ll = gumbel_tv_logl(psi, data)
    print("ll:", ll)
    assert np.isclose(ll, expected_ll, atol=1e-6)


def test_gumbel_tv():
    # Test with time-varying parameters
    theta = np.array([0.5, 0.2, 0.1])
    data = np.random.rand(100, 2)
    psi = gumbel_tv(theta, data)
    assert len(psi) == data.shape[0]
    assert np.min(psi) >= 1


if __name__ == "__main__":
    test_gumbel_tv_logl()
    test_gumbel_tv()

tau= [nan]
outs: [0.5] [0.6] [nan]
ll: nan


AssertionError: 

# Resources

* Here are some resources for further learning about dynamic copulas:

Book: "Dynamic Copulas for Finance" by Valentin Braun
Paper: "Dynamic Copulas for Monotonic Dependence Change in Time Series" by Changli He and Cheng Wang
Paper: "Dynamic Copula Models and High Frequency Data" by Irving De Lira Salvatierra and Andrew J. Patton
I hope this explanation of dynamic copulas is helpful. Please let me know if you have any further questions or would like me to elaborate on specific aspects of dynamic copulas and their applications in algorithmic trading.

* [Dynamic copula models and high frequency data] https://public.econ.duke.edu/~ap172/De_Lira_Salvatierra_Patton_JEmpFin_2015.pdf
* https://drawar.github.io/posts/dynamic-copula/
* https://www.alexandria.unisg.ch/server/api/core/bitstreams/eb7a384f-ea98-47d0-a971-e6a21447fccc/content
